# 08. Integration of Experimental Data

## Numerical experiments - Week 04/2025

_Boyan Mihaylov, MSc Computational Science (UVA/VU)_

## Prerequisite libraries

In [1]:
using Pkg
Pkg.add("CSV")

    Updating registry at `C:\Users\mihbo\.julia\registries\General.toml`
   Resolving package versions...
   Installed WorkerUtilities ─ v1.6.1
   Installed WeakRefStrings ── v1.4.2
   Installed CodecZlib ─────── v0.7.6
   Installed CSV ───────────── v0.10.15
    Updating `C:\Users\mihbo\.julia\environments\v1.10\Project.toml`
  [336ed68f] + CSV v0.10.15
    Updating `C:\Users\mihbo\.julia\environments\v1.10\Manifest.toml`
  [336ed68f] + CSV v0.10.15
  [944b1d66] + CodecZlib v0.7.6
  [ea10d353] + WeakRefStrings v1.4.2
  [76eceee3] + WorkerUtilities v1.6.1
┌ Warning: Circular dependency detected. Precompilation will be skipped for:
│   SpecialFunctionsExt [1285c0f1-ff9f-5867-b66e-0f359bcf09ba]
│   KernelAbstractions [63c18a36-062a-441e-b654-da1e3ab1ce7c]
│   CUDA [052768ef-5323-5732-b1bb-66c8b64840ba]
│   SparseArraysExt [85068d23-b5fb-53f1-8204-05c2aba6942f]
│   AtomixCUDAExt [13011619-4c7c-5ef0-948f-5fc81565cd05]
│   CUDAExt [11b7e2e0-d079-575b-885e-0ab22ef3252c]
│   LinearAlgebraExt 

In [6]:
using CSV

## 1. Reading and visualising germination data from Ijadpanahsaravi (2024)

There are two relevant data sets available from [1]. First, there are the results from incrementally increased inoculum sizes from 5000 to 40000 conidia per $150\mu\textrm{L}$ solution under different inducing carbon sources. A second series of experiments focuses on the two extremes of this conidial density range. The data from the `.csv` files is first parsed to extract two main measurements of interest - the incidence of swelling ($P_{\textrm{max}}$) and the corresponding half-saturation time $\tau$.

In [28]:
df_germination = DataFrame(CSV.File("Data/swelling_germination_results.csv"; header=true))
df_germination_extremes = DataFrame(CSV.File("Data/swelling_germination_results_extreme_densities.csv"))

println(df_germination[!, :"---"] == "Swelling")
df_germination_swelling = df_germination[df_germination[!, 1] == "Swelling"]
# df_germination

ErrorException: Invalid UTF-8 string

## References

- [1] Ijadpanahsaravi, Maryam, L. Basten Snoek, Wieke R. Teertstra and Han A. B. Wösten. “The impact of inter- and intra-species spore density on germination of the food spoilage fungus Aspergillus niger.” _International journal of food microbiology_ 410 (2023): 110495 .